In [1]:
from pyspark import SparkContext, SparkConf
import collections
import os
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk1.8.0_231"
os.environ["SPARK_HOME"] = "C:\\Users\\yizho\\Downloads\\spark-2.4.4-bin-hadoop2.7\\spark-2.4.4-bin-hadoop2.7"
os.environ['HADOOP_HOME'] = "C:\\winutils"
conf = SparkConf().setAppName("MLlib")
sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
spark = SparkSession(sc)

In [2]:
payment_df = spark.read.format("csv").option("header", "true").load(r'C:\Users\yizho\PycharmProjects\HW9\data\payment.csv')
payment_df = payment_df.alias("payment_df")
payment_df.show(3)
rental_df = spark.read.format("csv").option("header", "true").load(r'C:\Users\yizho\PycharmProjects\HW9\data\rental.csv')
rental_df = rental_df.alias('rental_df')
rental_df.show(3)
staff_df = spark.read.format("csv").option("header", "true").load(r'C:\Users\yizho\PycharmProjects\HW9\data\staff.csv')
staff_df = staff_df.alias('staff_df')
staff_df.show(3)
inventory_df = spark.read.format("csv").option("header", "true").load(r'C:\Users\yizho\PycharmProjects\HW9\data\inventory.csv')
inventory_df = inventory_df.alias('inventory_df')
inventory_df.show(3)
store_df = spark.read.format("csv").option("header", "true").load(r'C:\Users\yizho\PycharmProjects\HW9\data\store.csv')
store_df = store_df.alias('store_df')
store_df.show(3)
address_df = spark.read.format("csv").option("header", "true").load(r'C:\Users\yizho\PycharmProjects\HW9\data\address.csv')
address_df = address_df.alias('address_df')
address_df.show(3)
customer_df = spark.read.format("csv").option("header", "true").load(r'C:\Users\yizho\PycharmProjects\HW9\data\customer.csv')
customer_df = customer_df.alias('customer_df')
customer_df.show(3)
film_df = spark.read.format("csv").option("header", "true").load(r'C:\Users\yizho\PycharmProjects\HW9\data\film.csv')
film_df = film_df.alias('film_df')
film_df.show(3)
print("Homework solution start here follow by question order")

+----------+-----------+--------+---------+------+-------------------+-------------------+
|payment_id|customer_id|staff_id|rental_id|amount|       payment_date|        last_update|
+----------+-----------+--------+---------+------+-------------------+-------------------+
|         1|          1|       1|       76|  2.99|2005-05-25 11:30:37|2006-02-15 22:12:30|
|         2|          1|       1|      573|  0.99|2005-05-28 10:35:23|2006-02-15 22:12:30|
|         3|          1|       1|     1185|  5.99|2005-06-15 00:54:12|2006-02-15 22:12:30|
+----------+-----------+--------+---------+------+-------------------+-------------------+
only showing top 3 rows

+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|rental_id|        rental_date|inventory_id|customer_id|        return_date|staff_id|        last_update|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|        1|2005-05-2

In [3]:
payment_df.withColumn("amount", payment_df["amount"].cast("double"))
import pyspark.sql.functions as func
#payment_df.groupby('customer_id').agg(func.sum(payment_df.amount).alias('total_amount')).orderBy('total_amount',ascending = False).show(3)
payment_df.createOrReplaceTempView("payment_df")
spark.sql("select customer_id, sum(amount) as total_amount from payment_df group by customer_id order by total_amount DESC").show(3)

+-----------+------------------+
|customer_id|      total_amount|
+-----------+------------------+
|        526| 221.5500000000001|
|        148| 216.5400000000001|
|        144|195.58000000000007|
+-----------+------------------+
only showing top 3 rows



In [5]:
rental_df.createOrReplaceTempView("rental_df")
spark.sql("select * from rental_df where year(rental_date) = 2005 order by return_date").show(3)

+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|rental_id|        rental_date|inventory_id|customer_id|        return_date|staff_id|        last_update|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|       32|2005-05-25 04:06:21|        3832|        230|2005-05-25 23:55:21|       1|2006-02-15 21:30:53|
|       21|2005-05-25 01:59:46|         146|        388|2005-05-26 01:01:46|       2|2006-02-15 21:30:53|
|       14|2005-05-25 00:31:15|        2701|        446|2005-05-26 02:56:15|       1|2006-02-15 21:30:53|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
only showing top 3 rows



In [6]:
spark.sql("select staff_id, count(rental_id) as count from rental_df group by staff_id").show()

+--------+-----+
|staff_id|count|
+--------+-----+
|       1| 8040|
|       2| 8004|
+--------+-----+



In [7]:
staff_df.createOrReplaceTempView("staff_df")
spark.sql("select first(first_name) as first_name, first(last_name) as last_name, count(rental_id) as count from staff_df inner join rental_df on rental_df.staff_id = staff_df.staff_id group by staff_df.staff_id").show()

+----------+---------+-----+
|first_name|last_name|count|
+----------+---------+-----+
|      Mike|  Hillyer| 8040|
|       Jon| Stephens| 8004|
+----------+---------+-----+



In [8]:
inventory_df.createOrReplaceTempView("inventory_df")
store_df.createOrReplaceTempView("store_df")
address_df.createOrReplaceTempView("address_df")
customer_df.createOrReplaceTempView("customer_df")
film_df.createOrReplaceTempView("film_df")
spark.sql("""select r.rental_date, r.return_date, a.address, f.title, f.description, c.first_name, c.last_name, s.first_name, s.last_name
from rental_df as r
join inventory_df as i
on r.inventory_id = i.inventory_id
join store_df as st
on i.store_id = st.store_id
join address_df as a
on st.address_id = a.address_id
join customer_df as c
on r.customer_id = c.customer_id
join staff_df as s
on s.store_id = c.store_id
join film_df as f
on i.film_id = f.film_id
order by r.rental_date
 """).show(3)

+-------------------+-------------------+------------------+---------------+--------------------+----------+---------+----------+---------+
|        rental_date|        return_date|           address|          title|         description|first_name|last_name|first_name|last_name|
+-------------------+-------------------+------------------+---------------+--------------------+----------+---------+----------+---------+
|2005-05-24 22:53:30|2005-05-26 22:04:30| 47 MySakila Drive|BLANKET BEVERLY|A Emotional Docum...| CHARLOTTE|   HUNTER|      Mike|  Hillyer|
|2005-05-24 22:54:33|2005-05-28 19:40:33|28 MySQL Boulevard|   FREAKY POCUS|A Fast-Paced Docu...|     TOMMY|  COLLAZO|      Mike|  Hillyer|
|2005-05-24 23:03:39|2005-06-01 22:12:39|28 MySQL Boulevard|  GRADUATE LORD|A Lacklusture Epi...|    MANUEL|  MURRELL|      Mike|  Hillyer|
+-------------------+-------------------+------------------+---------------+--------------------+----------+---------+----------+---------+
only showing top 3 r

In [9]:
spark.sql("""select customer_id, total_amount from
(select customer_id, sum(amount) as total_amount from payment_df where year(payment_date) =2005 group by customer_id)
where total_amount > 200""").show()

+-----------+-----------------+
|customer_id|     total_amount|
+-----------+-----------------+
|        526|221.5500000000001|
|        148|216.5400000000001|
+-----------+-----------------+



In [10]:
customer_df.createOrReplaceTempView("customer_df")
spark.sql("""select * from
(select first(customer_df.first_name) as first_name, first(customer_df.last_name) as last_name, sum(amount) as total_amount from payment_df
inner join customer_df
on customer_df.customer_id = payment_df.customer_id where year(payment_date) =2005 group by payment_df.customer_id)
where total_amount > 200 """).show()

+----------+---------+-----------------+
|first_name|last_name|     total_amount|
+----------+---------+-----------------+
|      KARL|     SEAL|221.5500000000001|
|   ELEANOR|     HUNT|216.5400000000001|
+----------+---------+-----------------+



In [11]:
c2005 = spark.sql("select customer_id, year(rental_date) as rentyear from rental_df where year(rental_date) = 2005")
c2006 = spark.sql("select customer_id, year(rental_date) as rentyear from rental_df where year(rental_date) = 2006")
c2005.createOrReplaceTempView("c2005")
c2006.createOrReplaceTempView("c2006")
spark.sql("""select distinct(c2005.customer_id) as customer_id_in_2005_not_in_2006, c2005.rentyear from c2005
where c2005.customer_id not in (
    select c2006.customer_id from c2006
) """).show(10)
spark.sql("""select count(distinct(c2005.customer_id)) as customer_id_in_2005_not_in_2006 from c2005
where c2005.customer_id not in (
    select c2006.customer_id from c2006
) """).show()
spark.sql("""select distinct(c2006.customer_id) as customer_id_in_2006_not_in_2005, c2006.rentyear from c2006
where c2006.customer_id not in (
    select c2005.customer_id from c2005
) """).show(10)
spark.sql("""select count(c2006.customer_id) as customer_id_in_2006_not_in_2005 from c2006
where c2006.customer_id not in (
    select c2005.customer_id from c2005
) """).show(10)

+-------------------------------+--------+
|customer_id_in_2005_not_in_2006|rentyear|
+-------------------------------+--------+
|                            413|    2005|
|                            377|    2005|
|                            406|    2005|
|                            148|    2005|
|                            463|    2005|
|                             26|    2005|
|                            358|    2005|
|                             62|    2005|
|                            285|    2005|
|                            392|    2005|
+-------------------------------+--------+
only showing top 10 rows

+-------------------------------+
|customer_id_in_2005_not_in_2006|
+-------------------------------+
|                            441|
+-------------------------------+

+-------------------------------+--------+
|customer_id_in_2006_not_in_2005|rentyear|
+-------------------------------+--------+
+-------------------------------+--------+

+---------------------------